In [1]:
import numpy as np
import torch
import string
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.utils.rnn import pad_packed_sequence as unpack
from torch.nn.utils.rnn import pack_padded_sequence as pack
import math
import torch.utils.data as data
import json
import os
import pandas as pd
import random
import copy
import torch.utils.data.sampler as sampler
import torch.optim.lr_scheduler as lr_scheduler
import pickle

In [2]:
all_letters = string.printable
n_letters = len(all_letters)
def letterToIndex(letter):
    return all_letters.find(letter)
def linesToTensor(lines):
    line_length = 15000
    if max([ len(line) for line in lines]) < line_length:
        line_length = max( [len(line) for line in lines] )
    #line_length = max( [len(line) for line in lines] )
    #xx = [max(len(line)-15000,0) for line in lines]
    #print float(np.sum(xx)) / float(np.sum([len(line) for line in lines]))
    tensor = torch.zeros(len(lines), line_length, n_letters)
    for b, line in enumerate(lines): 
        line = line[:15000]
        for li, letter in enumerate(line):
            tensor[b][li + line_length - len(line)][letterToIndex(letter)] = 1 #뒤로 맞춰줌

    return tensor

In [22]:
class LangModel(nn.Module):
    def __init__(self, preTrained='True', input=100):
        super(LangModel, self).__init__()

        # Language Model
        self.lang = nn.LSTM(input, 128, 3, batch_first=True) 
 
        # Output 
        self.output = nn.Linear(128, 2)
        n = self.output.in_features * self.output.out_features
        self.output.weight.data.normal_(0, math.sqrt(2. / n))
        self.output.bias.data.zero_()

    def forward(self, text):
        text.cuda()
        h0 = ( Variable(torch.zeros(3, text.size(0), 128)).cuda(),  Variable(torch.zeros(3, text.size(0), 128)).cuda())

        lang_feature, hn = self.lang(text, h0 )
        lang_feature = lang_feature[:,-1,:]

        pred = self.output(lang_feature)
        return pred,lang_feature


In [4]:
import torch.utils.data as data

class chat_ds(data.Dataset):
    def __init__(self,d_type):
        self.gt_range =  1-0.25
        self.d_type=d_type
        with open('../multimodal/label/label.pickle','rb') as f1:  
            self.gt=pickle.load(f1)
        with open('./chat2.json','rb') as f2:  
            self.text=json.load(f2)
        #self.sample=list(map(int,list(self.gt.keys())))
        if d_type=='train':
            self.sample = ['102844412722519367','102844212429550795','102844401151219358','102844401154430631','102844412717014335','102844401153971877','102844224148503678','102844412722847048','102844401152857762','102844412707380528','102844212431516886','102844283027925085','102844412716227901','102844412710001974','102844294670878922','102844294670551241','102844283023599703','102844412704496937','102844235751783874','102844401152071328','102844412709674293','102844401153447587','102844224148896895','102844235746868664','102979081290790284','102844283027531868','102844212431975640','102844401155937960','102844212429092040','102844341906649746','102844412706987311','102844412721339716','102844212430402768','102844341905011343','102844235753356742','102844235750997440','102844412709346612','102844412705217835','102844235752963525','102844412712164667','102844412705545516','102844341912220311','102844341907370644','102844235749424575','102844212429419722','102844294669568199','102844212431779031','102844294666422466','102844224146472059','102844212428895431','102844212429747404','102844235748703677','102844224146930812','102844212430730450','102844294674876621','102844341909598870','102844283020453971','102844294670026952','102844412723174729','102844341904683662','102844283025696858','102844235747261881','102844401154168486','102844235748310460','102844412711836986','102844412723567946','102844235749031358','102844294674286796','102844294666881219','102844412716686654']
        if d_type=='val':
            self.sample = ['102844294671796427','102844224145685626','102844412717407552','102844235751390657','102844401156069033','102904869420860038','102910307641576395','102844341905404560','102844341906977427','102844212430075086','102844412711116088','102844401153578660','102844294667405508','102844412706659630']
        if d_type=='test':
            self.sample = ['102844212431058132','102844341902586509','102844401152267937','102844212430927059','102844412708953395','102844212429944013','102844341912679064','102844235753749959','102844341908026005','102844283023206486','102844224147717245','102844412704890154','102844212430599377','102844412711443769','102844235747982779']
            
        self.WeightedSampling=[]
        for i in self.sample:
            self.WeightedSampling.extend(copy.copy(self.gt[str(i)]))
        
        sampling = np.array(self.WeightedSampling)
        neg_idx = np.where(sampling == 0)[0] #general
        pos_idx = np.where(sampling == 1)[0] #highlight
        sampling = sampling.astype(np.float32)

        begin_pos = 0 
        hl_frames = []
        for it, cur_pos in enumerate(pos_idx):
            if it+1 < len(pos_idx): 
                if((pos_idx[it+1] - cur_pos) > 1):#cur_pos와 cur_pos+1 간격이 1보다 크면, 즉 다른 구간이면
                    begin = int((it+1 - begin_pos) * self.gt_range) + begin_pos
                    hl_frames.extend( pos_idx[begin: it] ) #한구간의 하이라이트 25%만 사용하겠다.
                    begin_pos = it+1


        sampling.fill(0)
        sampling[neg_idx] = len(sampling) / float(len(neg_idx))
       # self.WeightedSampling[pos_idx] = len(self.WeightedSampling) / float(len(pos_idx))
        sampling[hl_frames] = len(sampling) / float(len(hl_frames))
        self.WeightedSampling = sampling

        self.sum=np.insert(np.cumsum([len(self.gt[str(i)]) for i in self.sample]),0,0)
        print("data load fin")

        
    def __len__(self):
        return len(self.WeightedSampling)
    def __getitem__(self,index):
            vid=np.histogram(index,self.sum)
            vid = np.where(vid[0]>0)[0][0]
            vframe=index-self.sum[vid]
            game_id=str(self.sample[vid])


            win_text=''
            for idx in range(7): #7 : window size
                if vframe+idx<len(self.text[str(game_id)]):
                    win_text+=self.text[str(game_id)][vframe+idx]+'\n'

            label=self.gt[str(game_id)][vframe]
            return win_text,label,str(game_id)
        
            
            



In [5]:
class SampleSequentialSampler(sampler.Sampler):
    """Samples elements sequentially, always in the same order.
    Arguments:
        data_source (Dataset): dataset to sample from
        offset (int): offset between the samples
    """

    def __init__(self, data_source, offset=10):
        self.num_samples = len(data_source) 
        self.offset = offset

    def __iter__(self):
        return iter(np.arange(0, self.num_samples, self.offset ))

    def __len__(self):
        return len(np.arange(0, self.num_samples, self.offset ))


In [6]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    if epoch<20:
        lr = 0.01
    else:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

    return lr

In [7]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [8]:
def fmeasure(output, target):
    _, pred = output.topk(1, 1, True, True)
    pred = pred.view(-1,1)
    target = target.view(-1,1)

    #overlap = ((pred== 1) + (target == 1)).gt(1)
    #overlap = overlap.view(-1,1)
    TP = len(np.where((pred==1)&(target==1)==True)[0]) # True positive
    FP = len(np.where((pred==1)&(target==0)==True)[0]) # Condition positive = TP + FN
    TN = len(np.where((pred==0)&(target==0)==True)[0])
    FN = len(np.where((pred==0)&(target==1)==True)[0])

    
    #overlap_len = overlap.data.long().sum()
    pred_len = pred.data.long().sum()
    gt_len   =  target.data.long().sum()

    return TP,FP,TN,FN,pred_len, gt_len

In [9]:
def clip_gradient(model, clip_norm):
    """Computes a gradient clipping coefficient based on gradient norm."""
    totalnorm = 0
    for pm in model.parameters():
        if pm.requires_grad:
            if pm.grad is not None:
                modulenorm = pm.grad.data.norm()
                totalnorm += modulenorm ** 2

    totalnorm = np.sqrt(totalnorm.cpu())
    norm = clip_norm / max(totalnorm, clip_norm)
    for p in model.parameters():
        if p.requires_grad:
            if p.grad is not None:
                p.grad.mul_(norm)

In [10]:
#game_id='102844212428895431'
train=chat_ds('train')
val=chat_ds('val')

data load fin
data load fin


In [11]:
sampler1 = torch.utils.data.sampler.WeightedRandomSampler(weights=train.WeightedSampling.tolist(), num_samples=10000)
train_loader=torch.utils.data.DataLoader(train,batch_size=32,sampler=sampler1)
sampler2 =  SampleSequentialSampler(val, 30)
val_loader=torch.utils.data.DataLoader(val,batch_size=32,sampler= sampler2)

In [23]:
###### model load #####
model=LangModel().cuda()
criterion = nn.CrossEntropyLoss().cuda()

In [13]:
weight_dir='./new_label_final data/'

In [ ]:
#train set

if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)
optimizer = torch.optim.SGD(model.parameters(), 0.01,momentum=0.9,weight_decay=1e-4)
# scheduler = lr_scheduler.ReduceLROnPlateau(optimizer)

with open(weight_dir+'train_result','a') as f:
    f.write('=========_ReduceLROnPlateau  ================\n')


best_losses=1000000
for epoch in range(60):
    lr = adjust_learning_rate(optimizer, epoch)
    train_loss=0
    print(epoch)
    model.train()
    for i, (text,labels,game_id) in enumerate(train_loader):
        inputs=linesToTensor(text)
        inputs = Variable(inputs).cuda()
        labels = Variable(labels).cuda()
        optimizer.zero_grad()
        output=model(inputs)

        loss=criterion(output,labels.long())
        train_loss+=loss
        #torch.nn.utils.clip_grad_norm_(model.parameters(), 1.) #rnn 계열 gradient 장치
        loss.backward()
        optimizer.step()


    #validation
    model.eval()
    val_loss=0
    acc=0
    gt_sum=0
    with open(weight_dir+'train_result','a') as f:
        with torch.no_grad():
            for it, (text,labels,game_id) in enumerate(val_loader):
                inputs=linesToTensor(text)
                inputs = Variable(inputs).cuda()
                labels = Variable(labels).cuda()
                output=model(inputs)
                loss=criterion(output,labels.long())
                val_loss+=loss
                TP,FP,TN,FN,pred_len, gt_len=fmeasure(output.cpu(),labels.cpu())
                acc=acc+TP+TN
                gt_sum+=len(output)
            val_acc=acc/gt_sum
            print("epoch {} train_loss : {} , val_loss : {}, val_acc : {}".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader),val_acc))
            f.write("epoch {} train_loss : {} , val_loss : {}, val_acc : {}\n".format(epoch,train_loss/len(train_loader),val_loss/len(val_loader),val_acc))
            if best_losses>val_loss:
                best_losses=val_loss
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train_best"))
                f.write("epoch {} saved\n".format(epoch))
            else:
                torch.save(model.state_dict(),'{}'.format(weight_dir+str(epoch)+"train"))




0
epoch 0 train_loss : 0.69375079870224 , val_loss : 0.7068176865577698, val_acc : 0.17386363636363636
1
epoch 1 train_loss : 0.6940241456031799 , val_loss : 0.7162960171699524, val_acc : 0.17386363636363636
2
epoch 2 train_loss : 0.6935345530509949 , val_loss : 0.6806250214576721, val_acc : 0.8261363636363637
3
epoch 3 train_loss : 0.693516194820404 , val_loss : 0.6973260641098022, val_acc : 0.17386363636363636
4
epoch 4 train_loss : 0.6933884024620056 , val_loss : 0.6844025254249573, val_acc : 0.8261363636363637
5
epoch 5 train_loss : 0.6935926079750061 , val_loss : 0.7162016034126282, val_acc : 0.17386363636363636
6
epoch 6 train_loss : 0.6931782364845276 , val_loss : 0.711101770401001, val_acc : 0.17386363636363636
7


In [19]:
#test

test=chat_ds('test')
test_loader=torch.utils.data.DataLoader(test,batch_size=32)
dataset=weight_dir+'14train_best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.cuda()
model.eval()
pred_sum = 0#model output
gt_sum = 0#label
tp_sum=0
fp_sum=0
fn_sum=0
acc=0
sum=0
result={}
with torch.no_grad():
    for it, (text,labels,g) in enumerate(test_loader):
        inputs=linesToTensor(text)
        inputs = inputs.cuda()
        labels = labels.cuda()
        output=model(inputs)

        TP,FP,TN,FN,pred_len, gt_len=fmeasure(output.cpu(),labels.cpu())
        print(TP,FP,TN,FN,pred_len, gt_len)
        tp_sum += TP
        fp_sum += FP
        fn_sum += FN
        pred_sum += pred_len
        gt_sum += gt_len
        acc=acc+TP+TN
        sum+=len(output)
    with open(weight_dir+'/train_result','a') as f:
        if tp_sum>0 and fp_sum>0 and fn_sum>0:
            precision = tp_sum/(tp_sum+fp_sum)
            recall = tp_sum / (tp_sum+fn_sum)
            f1 = (2*precision*recall / (precision + recall)) * 100
            accuracy=acc/sum
            print( tp_sum, fp_sum, fn_sum)
            print('[{}/{}], prec:{}, recall:{}, f1:{}, acc: {}'.format(it, len(test_loader), precision, recall, f1,accuracy))
            f.write('{}, prec:{}, recall:{}, f1:{}, acc : {}\n'.format(dataset, precision, recall, f1,accuracy))

data load fin
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 27 5 tensor(0) tensor(5)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 16 16 tensor(0) tensor(16)
0 0 15 17 tensor(0) tensor(17)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 20 12 tensor(0) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 25 7 tensor(0) tensor(7)
0 0 14 18 tensor(0) tensor(18)
0 0 24 8 tensor(0) tensor(8)
0 0 32 0 tensor(0) tensor(0)
0 0 19 13 tensor(0) t

0 0 22 10 tensor(0) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 15 17 tensor(0) tensor(17)
0 0 19 13 tensor(0) tensor(13)
0 0 22 10 tensor(0) tensor(10)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 18 14 tensor(0) tensor(14)
0 0 21 11 tensor(0) tensor(11)
0 0 22 10 tensor(0) tensor(10)
0 0 30 2 tensor(0) tensor(2)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 1 16 15 tensor(1) tensor(15)
0 0 23 9 tensor(0) tensor(9)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 29 3 tensor(0) tensor(3)
0 0 0 32 tensor(0) tensor(32)
0 0 12 20 tensor(0) tensor(20)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) tensor(0)
0 0 3 29 tensor(0) tensor(29)
2 1 19 10 tensor(3) tensor(12)
0 0 32 0 tensor(0) tensor(0)
0 0 32 0 tensor(0) te

KeyboardInterrupt: 

In [ ]:
#test

test=chat_ds('train')
test_loader=torch.utils.data.DataLoader(test,batch_size=1)
dataset=weight_dir+'14train_best'
checkpoint=torch.load(dataset,map_location='cuda:0')
model.load_state_dict(checkpoint)
model.cuda()
model.eval()

result={}
result2={}
result3={}
with torch.no_grad():
    for it, (text,labels,game_id) in enumerate(test_loader):
        inputs=linesToTensor(text)
        inputs = inputs.cuda()
        labels = labels.cuda()
        output_p,output_f=model(inputs)
        output=(output)
        _, pred = output_p.topk(1, 1, True, True)
        pred = pred.view(-1,1)
        if game_id[0] not in result.keys():
            print(game_id[0])
            result[game_id[0]]=[int(pred[0][0])]
            result2[game_id[0]]=[(output_f[0]).tolist()]
            result3[game_id[0]]=[(output_p[0]).tolist()]
            
        else:
            result[game_id[0]].append(int(pred[0][0]))
            result2[game_id[0]]+=[(output_f[0]).tolist()]
            result3[game_id[0]]+=[(output_p[0]).tolist()]

        



data load fin
102844412722519367
102844212429550795
102844401151219358
102844401154430631
102844412717014335
102844401153971877
102844224148503678
102844412722847048
102844401152857762
102844412707380528
102844212431516886
102844283027925085
102844412716227901
102844412710001974
102844294670878922
102844294670551241
102844283023599703
102844412704496937
102844235751783874
102844401152071328
102844412709674293
102844401153447587
102844224148896895
102844235746868664
102979081290790284
102844283027531868
102844212431975640
102844401155937960
102844212429092040
102844341906649746
102844412706987311
102844412721339716
102844212430402768
102844341905011343
102844235753356742
102844235750997440
102844412709346612
102844412705217835
102844235752963525
102844412712164667
102844341907370644


In [37]:
with open(weight_dir+'chat_bin_train.json','a') as f:
    json.dump(result,f)
with open(weight_dir+'chat_prob_train.json','a') as f:
    json.dump(result3,f)
with open(weight_dir+'chat_feature_train.json','a') as f:
    json.dump(result2,f)

In [128]:
x=torch.tensor([[ 0.0124, -0.0438, -0.0314, -0.0224,  0.0108, -0.0189,  0.0026,  0.0232,
         -0.0182, -0.0174,  0.0452,  0.0375, -0.0213,  0.0411, -0.0517,  0.0155,
          0.0682, -0.0051, -0.0309, -0.0049,  0.0018, -0.0063,  0.0157,  0.0178,
          0.0045,  0.0187, -0.0101,  0.0066,  0.0157, -0.0387,  0.0258, -0.0338,
          0.0075,  0.0497,  0.0151,  0.0546,  0.0097, -0.0002,  0.0450,  0.0119,
         -0.0144,  0.0214,  0.0165,  0.0134,  0.0420,  0.0768, -0.0200,  0.0237,
          0.0432,  0.0226,  0.0176,  0.1564,  0.0149, -0.0334,  0.0200,  0.0510,
          0.0013, -0.0187, -0.0430,  0.0499,  0.0285,  0.0552, -0.0237,  0.0297,
          0.0055, -0.0049, -0.0273,  0.0123, -0.0218, -0.0252, -0.0567, -0.0266,
          0.0108, -0.0038,  0.0411,  0.0293,  0.0558, -0.0290, -0.0112, -0.0141,
          0.0392, -0.0739,  0.0045, -0.0106, -0.0030,  0.0166,  0.0260,  0.0099,
          0.0313,  0.0104,  0.0105,  0.0058,  0.0141, -0.0126, -0.0301,  0.0465,
         -0.0182, -0.0304, -0.0034,  0.0460, -0.0364, -0.0233,  0.0174, -0.0238,
          0.0647,  0.0091, -0.0461,  0.0063,  0.0016, -0.0056,  0.0563,  0.0176,
         -0.0710,  0.0373, -0.0593, -0.0405,  0.0052, -0.0136,  0.0320,  0.0190,
          0.0060, -0.0392, -0.0333, -0.0166,  0.0437,  0.0176, -0.0003,  0.0078]],
       device='cuda:0')

In [169]:
a=[x[0].tolist()]

In [171]:
a.append(x[0].tolist())

In [34]:
result['102844224148503678']

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [27]:
len(result['102844235753356742'])

1654

In [17]:
x['102844235753356742']

NameError: name 'x' is not defined

In [24]:
len(result.keys())

60

In [25]:
            sample =[102844235753356742,102844294666422466, 102844341906256529, 102979081290790284, 102844212430271695, 102844401155937960, 102844412704496937, 102844412722519367, 102844235749031358, 102844412709674293, 102844341902586509, 102844235750997440, 102844212431975640, 102844294667995333, 102844341912220311, 102844235747261881, 102844294670551241, 102844412707708209, 102844212430599377, 102844224145685626, 102844401152857762, 102844341908026005, 102844341909598870, 102844283023599703, 102844235753749959, 102844294670026952, 102844212430075086, 102844283027531868, 102844212430927059, 102844212429419722, 102844283025696858, 102844224146472059, 102844412712164667, 102844401153971877, 102844235748310460, 102844412711116088, 102844235752111555, 102844283020453971, 102844294666881219, 102844401152267937, 102844212429944013, 102844294671796427, 102844401151874719, 102844412723567946, 102844294669568199, 102844412709346612, 102844212431779031, 102844212430402768, 102844412711443769, 102844283027925085, 102844235746868664, 102844283023206486, 102844401154168486, 102844212429550795, 102844341907370644, 102844412721339716, 102844294667405508, 102844294674876621, 102844212429288649, 102844401154430631]


In [26]:
len(sample)

60

In [50]:
with open(weight_dir+'chat_128_test.json','rb') as f:
    r=json.load(f)

In [61]:
r['102844412717014335'][:10]

[[0.03537170588970184,
  -0.043081168085336685,
  -0.04096755012869835,
  -0.015679210424423218,
  -0.000730052764993161,
  -0.039760321378707886,
  -0.01698194444179535,
  0.03368096053600311,
  -0.030482346192002296,
  -0.05684709921479225,
  0.07552143186330795,
  0.05213841795921326,
  -0.06367486715316772,
  0.03364301472902298,
  -0.09859336912631989,
  0.020916694775223732,
  0.0760808065533638,
  -0.013417842797935009,
  -0.05708283558487892,
  -0.0011524142464622855,
  0.0006507260841317475,
  -0.011696960777044296,
  0.024534426629543304,
  0.024831000715494156,
  0.013955054804682732,
  0.010192235000431538,
  -0.019946249201893806,
  0.008109033107757568,
  0.005184138659387827,
  -0.05684118717908859,
  0.03394879773259163,
  -0.04553905874490738,
  0.009822734631597996,
  0.05076994746923447,
  0.01525385957211256,
  0.05976983159780502,
  0.03937987610697746,
  0.025555623695254326,
  0.05062054097652435,
  0.0014553380897268653,
  -0.033140629529953,
  0.045567929744720

In [54]:
r['102844412710001974'][:10]

[[0.021551288664340973,
  -0.04384525120258331,
  -0.03462623432278633,
  -0.02016858384013176,
  0.006743958685547113,
  -0.028185468167066574,
  -0.004008247051388025,
  0.026203304529190063,
  -0.024540366604924202,
  -0.03245027735829353,
  0.055668849498033524,
  0.041380468755960464,
  -0.036310501396656036,
  0.03796372935175896,
  -0.06945107877254486,
  0.018607456237077713,
  0.07178831845521927,
  -0.008162139914929867,
  -0.040763359516859055,
  -0.0024862613063305616,
  0.00035980346729047596,
  -0.008969173766672611,
  0.020026447251439095,
  0.020622877404093742,
  0.007058870047330856,
  0.014689539559185505,
  -0.012295215390622616,
  0.008632087148725986,
  0.011394845321774483,
  -0.04461101070046425,
  0.03048529289662838,
  -0.04071923717856407,
  0.008205780759453773,
  0.049318742007017136,
  0.015584717504680157,
  0.057505540549755096,
  0.021161774173378944,
  0.01125938631594181,
  0.04544339329004288,
  0.0056217750534415245,
  -0.023245401680469513,
  0.031

In [38]:
len(result.keys())

70